<a href="https://colab.research.google.com/github/mathluva/bert-as-embedder/blob/main/Bert_as_embedder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#import dependencies
import numpy as np
import math
import re
import pandas as pd
from bs4 import BeautifulSoup
import random

from google.colab import drive

In [2]:
#use ! for terminal commands
!pip install bert-for-tf2 #tensorflow2 light version
!pip install sentencepiece #required for BERT-tf2

     |████████████████████████████████| 51kB 7.1MB/s 
  Created wheel for bert-for-tf2: filename=bert_for_tf2-0.14.9-cp37-none-any.whl size=30535 sha256=638a48c44e0022722c64a2d92a0a965bb6710d8f9c1fd64b1bf63f256d13f302
  Stored in directory: /root/.cache/pip/wheels/a1/04/ee/347bd9f5b821b637c76411d280271a857aece00358896a230f
  Created wheel for py-params: filename=py_params-0.10.2-cp37-none-any.whl size=7912 sha256=40315e52bf766149f39b6468cf6bba59d2794abdbe4c3deb4a9102cb18171b89
  Stored in directory: /root/.cache/pip/wheels/d0/4a/70/ff12450229ff1955abf01f365051d4faae1c20aef53ab4cf09
  Created wheel for params-flow: filename=params_flow-0.8.2-cp37-none-any.whl size=19472 sha256=3e23e7c862e96d731eda46fd693249c0e59b78b1afbe14a6773503adb8c2ce7d
  Stored in directory: /root/.cache/pip/wheels/08/c8/7f/81c86b9ff2b86e2c477e3914175be03e679e596067dc630c06
Successfully built bert-for-tf2 py-params params-flow
     |████████████████████████████████| 1.2MB 19.8MB/s 


In [3]:
try:
    %tensorflow_version 2.x #only available in Google colab
except Exception:
    pass
import tensorflow as tf

import tensorflow_hub as hub #used to import the weights from BERT

from tensorflow.keras import layers
import bert #installed in previous step

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `2.x #only available in Google colab`. This will be interpreted as: `2.x`.


TensorFlow 2.x selected.


In [4]:
#load files, data preprocessing
drive.mount("/content/drive")

Mounted at /content/drive


In [5]:
#label columns
#latin1 is common for western languages
cols = ["sentiment", "id", "date", "query", "user", "text"]
data = pd.read_csv(
    "/content/drive/MyDrive/trainingandtestdata.zip (Unzipped Files)/training.1600000.processed.noemoticon.csv", 
    header = None,
    names = cols,
    engine = "python",
    encoding = "latin1")

In [6]:
#axis1 column data
#without inplace=True, it would be required to write data = data.drop("...")
data.drop(["id", "date","query", "user"], axis = 1, inplace = True)

In [7]:
data.head()

,sentiment,text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."


In [8]:
#cleaning
#r is regEX (regexr.com for more documentation)
def clean_tweet(tweet):
    tweet = BeautifulSoup(tweet, "lxml").get_text() #tweets are not usuable as standard string, need BS to extract string
    tweet = re.sub(r"@[A-Za-z0-9]+", ' ',tweet)#anything behind @symbol with empty space, apply to tweet
    tweet = re.sub(r"https?://[A-Za-z0-9./]+", ' ', tweet)#? means the s can be there or not
    tweet = re.sub(r"[^a-zA-Z.!?]", ' ', tweet) #keep only standard characters
    tweet = re.sub(r" +", ' ', tweet) #replace multiple sequences of white space with only one white space
    return tweet

In [9]:
data_clean = [clean_tweet(tweet) for tweet in data.text]

In [10]:
#process sentiment
data_labels = data.sentiment.values
data_labels[data_labels ==4] =1 #data is using 0 and 4, replace 4 with standard 1

In [28]:
#create BERT layer to have access to metadata for the tokenizer(like vocab size).
#call BERT as a layer, hub is where all pretrained models are located
#trainable = False bc we are not fine-tuning the weights
FullTokenizer = bert.bert_tokenization.FullTokenizer
bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",
                            trainable = False) 
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy() #way to have acces to vocab
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = FullTokenizer(vocab_file, do_lower_case)

We only use the first sentence for BERT inputs so we add the CLS token at the beginning and the SEP token at the end of each sentence.

In [40]:
#the first layer is a BERT layer so
#make inputs suitable for BERT
def encode_sentence(sent):
    return ["CLS"] + tokenizer.tokenize(sent) + ["SEP"]


In [ ]:
data_inputs = [encode_sentence(sentence) for sentence in data_clean]

Dataset Creation

We need to create 3 different inputs for each sentence:
1.  The tokenize version of the sentence created in data_inputs
2. Mask layer: 1 for standard tokens and 0 for padding
3. Segment input: sequence of 0 and 1's , 0  correspond to first sentence and 1 for second sentence 

In [ ]:
def get_ids(tokens):
    return tokenizer.convert_ids_to_tokens(tokens)


#padding mask, compare each element of token with the string "[PAD]"
#not_equal will give us a 1 when not using [PAD] token
def get_mask(tokens):
    return np.char.not_equal(tokens, "[PAD]").astype(int)


#determine if 1st or 2nd sentence
#first sentence is 0 "current_id", after [SEP] turn 0 to 1
def get_segments(tokens):
    seg_ids = []
    current_seg_id = 0
    for tok in tokens:
        seg_ids.append(current_seg_id)
        if tok =="[SEP]":
            current_seg_id = 1- current_seg_id  #once we get to a second [SEP] it will turn back to 0, for sentence pair options in BERT
        return seg_ids

We will create padded batches (so we pad sentences for each batch independently), this way we add the minimum of padding tokens possible.  For that, we sort sentences by length, apply padded_batches and then shuffle.

In [ ]:
data_with_len = [[sent, data_labels[i], len(sent)] for i, sent in enumerate(data_inputs)]
random.shuffle(data_with_len)
data_with_len.sort(key = lambda x:x[2]) #use 3rd element to sort list
sorted_all = [([get_ids(sent_lab[0]), 
              get_mask(sent_lab[0]),
              get_segments(sent_lab[0])],
               sent_lab[1]) for sent_lab in data_with_len if sent_lab[2] >7]